In [57]:
from collections import Counter

#functions

# buy product a -> x get one for y % of the price (examples: 2 and the third would be 50%, 1 and the second would be 100%)
# CEO: GR1 + GR1 = 3.11 + 0
# COO: SR1 + SR1 + SR1 = 4.50 + 4.50 + 4.50
# VP: 11.23 + 11.23 + 11.23 = 11.23 x 3/2 + 11.23 x 3/2 + 11.23 x 3/2

# Three cases: 
# case 1: after x amount get y for free (here you have to use mod I think)
# case 2: after x amount each price changes to given price y.
# case 3: after x amount each price changes by a friction of original price.


# case 1

def free_product_included(discount_dict, count_occurances, promo_amount = 1, promo_free = 1):
    
    discount_product_list = [product for product, discount in discount_dict.items() if discount == "free_product_included"]
    must_pay_quantities = count_occurances.copy()

    for discount_product in discount_product_list:
        if discount_product in count_occurances:
            free_items = (count_occurances[discount_product] // (promo_amount + promo_free)) * promo_free
            must_pay_quantities[discount_product] = count_occurances[discount_product] - free_items

    return must_pay_quantities

In [64]:
# tests for case 1
import unittest

class TestFreeProductIncluded(unittest.TestCase):
    
    def test_buy_1_get_1_free(self):
        final_cashier_list = ['GR1', 'GR1', 'GR1', 'GR1', 'GR1', 'SR1']
        count_occurances = dict(Counter(final_cashier_list))
        global discount_dict 
        discount_dict = {"GR1": "free_product_included"} 
        self.assertEqual(free_product_included(count_occurances, 1, 1), {'GR1': 3, 'SR1': 1})

    def test_buy_2_get_1_free(self):
        final_cashier_list = ['GR1', 'GR1', 'GR1', 'GR1', 'GR1', 'SR1']
        count_occurances = dict(Counter(final_cashier_list))
        global discount_dict
        discount_dict = {"GR1": "free_product_included"} 
        self.assertEqual(free_product_included(count_occurances, 2, 1), {'GR1': 4, 'SR1': 1})

    def test_buy_5_get_1_free(self):
        final_cashier_list = ['GR1', 'GR1', 'GR1', 'GR1', 'GR1', 'SR1']
        count_occurances = dict(Counter(final_cashier_list))
        global discount_dict
        discount_dict = {"GR1": "free_product_included"} 
        self.assertEqual(free_product_included(count_occurances, 5, 1), {'GR1': 5, 'SR1': 1})

    def test_product_not_in_list(self):
        final_cashier_list = ['SR1']
        count_occurances = dict(Counter(final_cashier_list))
        global discount_dict
        discount_dict = {"GR1": "free_product_included"} 
        self.assertEqual(free_product_included(count_occurances, 5, 1), {'SR1': 1})

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

....
----------------------------------------------------------------------
Ran 4 tests in 0.003s

OK
